In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy
import time
from scipy import ndimage
from scipy.interpolate import griddata
import os

import TileUtility as TU

%matplotlib ipympl

ModuleNotFoundError: No module named 'ipympl'

In [ ]:
HT = TU.HalfTiles(TU.FullTiles())

In [ ]:
# based on code from MP1 utils.py
def prompt_eye_selection(image):
    fig = plt.figure()
    plt.imshow(image, cmap='gray')
    fig.set_label('Click on corners of game board')
    plt.axis('off')
    xs = []
    ys = []
    clicked = np.zeros((4, 2), dtype=np.float32)

    # Define a callback function that will update the textarea
    def onmousedown(event):
        x = event.xdata
        y = event.ydata
        xs.append(x)
        ys.append(y)

        plt.plot(xs, ys, 'r-+')

    def onmouseup(event):
        if(len(xs) >= 4):
            plt.plot(xs ++ xs[0], ys ++ ys[0], 'r-+')
            plt.close(fig)

    def onclose(event):
        clicked[:, 0] = xs
        clicked[:, 1] = ys
    # Create an hard reference to the callback not to be cleared by the garbage
    # collector
    fig.canvas.mpl_connect('button_press_event', onmousedown)
    fig.canvas.mpl_connect('button_release_event', onmouseup)
    fig.canvas.mpl_connect('close_event', onclose)

    return clicked

In [ ]:
def fineTunePoint(image, point):
    fig = plt.figure()
    
    Z = np.array([[point[0] - 50, point[1] - 50], [point[0] + 50, point[1] + 50]]).astype(int)
    plt.imshow(image[Z[0,1]:Z[1,1], Z[0,0]:Z[1,0]])
    plt.plot(50, 50, 'b-+')
    
    fig.set_label('Fine Tune Corner Selection')
    plt.axis('off')
    clicked = point
    hasClicked = False
    x = []
    y = []
    
    # Define a callback function that will update the textarea
    def onmousedown(event):
        x.append(event.xdata)
        y.append(event.ydata)
        hasClicked = True
        clicked[0] = x[0] + point[0] - 50
        clicked[1] = y[0] + point[1] - 50
        print("clicked is now {0}".format(clicked))
        plt.plot(x, y, 'r-+')

    def onmouseup(event):
        plt.plot(x, y, 'r-+')

    def onclose(event):
        pass
    
    # Create an hard reference to the callback not to be cleared by the garbage
    # collector
    fig.canvas.mpl_connect('button_press_event', onmousedown)
    fig.canvas.mpl_connect('button_release_event', onmouseup)
    fig.canvas.mpl_connect('close_event', onclose)

    return clicked

In [ ]:
img = cv2.cvtColor(cv2.imread("dev_images/20201123_132903.jpg"), cv2.COLOR_BGR2RGB)
points = prompt_eye_selection(img)

In [ ]:
points2 = points.copy()

In [ ]:
points = points2.copy()

In [ ]:
points[0] = fineTunePoint(img, points[0])

In [ ]:
points

In [ ]:
points[1] = fineTunePoint(img, points[1])

In [ ]:
points[2] = fineTunePoint(img, points[2])

In [ ]:
points[3] = fineTunePoint(img, points[3])

In [ ]:
points = np.array([[ 501.51974, 2562.663  ],
       [ 931.8444 , 1369.1488 ],
       [1986.8228 ,  993.16016],
       [1870.762  , 2285.566  ]])

In [ ]:
# Crop the image to enclose boundary
cropX = (int(np.min(points[:,0])), int(np.max(points[:,0])))
cropY = (int(np.min(points[:,1])), int(np.max(points[:,1])))

board = img.copy()
board = board[cropY[0]:cropY[1], cropX[0]:cropX[1]]
plt.imshow(board)

points_cropped = [ [ x - cropX[0], y - cropY[0] ] for x, y in points ]
print(points_cropped)

In [ ]:
# This setup follows 
# http://www.cse.psu.edu/~rtc12/CSE486/lecture16.pdf

tile_res = 32

cx = [ p[0] for p in points_cropped ]
cy = [ p[1] for p in points_cropped ]
ix = [ 0, 5*tile_res, 5*tile_res, 0 ]
iy = [ 0, 0, 5*tile_res, 5*tile_res ]

# the following setups mapping of rhombus identified above to a rectangle with corners (0,0) and (2,1)
T1 = np.array([
    [cx[0], cy[0], 1.,     0,     0,  0, -cx[0]*ix[0], -cy[0]*ix[0]],
    [    0,     0,  0, cx[0], cy[0],  1, -cx[0]*iy[0], -cy[0]*iy[0]],
    [cx[1], cy[1], 1.,     0,     0,  0, -cx[1]*ix[1], -cy[1]*ix[1]],
    [    0,     0,  0, cx[1], cy[1],  1, -cx[1]*iy[1], -cy[1]*iy[1]],
    [cx[2], cy[2], 1.,     0,     0,  0, -cx[2]*ix[2], -cy[2]*ix[2]],
    [    0,     0,  0, cx[2], cy[2],  1, -cx[2]*iy[2], -cy[2]*iy[2]],
    [cx[3], cy[3], 1.,     0,     0,  0, -cx[3]*ix[3], -cy[3]*ix[3]],
    [    0,     0,  0, cx[3], cy[3],  1, -cx[3]*iy[3], -cy[3]*iy[3]]])

c1 = np.array([ix[0], iy[0], ix[1], iy[1], ix[2], iy[2], ix[3], iy[3]])

est = np.linalg.solve(T1, c1)
A = np.array([
    [est[0], est[1], est[2]],
    [est[3], est[4], est[5]],
    [est[6], est[7], 1.0]])
Ainv = np.linalg.inv(A)


print(est)
print(A)
print(Ainv)

In [ ]:
def invMap(xs, ys):
    txs = Ainv[0, 0]*xs + Ainv[0, 1]*ys + Ainv[0, 2]
    tys = Ainv[1, 0]*xs + Ainv[1, 1]*ys + Ainv[1, 2]
    hs = Ainv[2, 0]*xs + Ainv[2, 1]*ys + Ainv[2, 2]
    txs = txs / hs
    tys = tys / hs
    return txs, tys

In [ ]:
test = invMap(np.array([1, 2, 2, 1]), np.array([0, 0, 1, 1]))
plt.scatter(test[0], test[1])

In [ ]:
ys, xs = np.mgrid[0:(5*tile_res), 0:(5*tile_res)]
txs, tys = invMap(xs, ys)
old_grid_y, old_grid_x = np.mgrid[0:board.shape[0], 0:board.shape[1]]

In [ ]:
result = np.zeros((5*tile_res, 5*tile_res, 3))

for c in range(3):
    test = griddata(
        np.stack((old_grid_x, old_grid_y)).transpose().reshape(-1, 2),
        board[:,:,c].T.flatten(),
        (txs, tys))
    result[:,:,c] = test

In [ ]:
plt.imshow(result.astype('uint8'))

In [ ]:
plt.imshow(result[100:200, 100:200].astype('int'))

In [ ]:
np.histogram(result[100:200, 100:200].astype('int'))

In [ ]:
plt.imshow(board)

In [ ]:
result.shape

In [ ]:
cv2.imwrite("test.png", result)